## ❓ Pre-module quiz

Why is Naive Bayes "naive"?

A. Because it's the most basic, i.e. "naive" classifier we can build

B. Because it "naively" assumes that the probabilities of features (i.e., in our case, words) are independent of each other

C. Because the guy who invented it tought it was a cool name

D. Because it "naively" assumes that the probabilities of features (i.e., in our case, words) are dependent of each other

<hr>    <!-- please remember this! -->
<details>
  <summary>Click <b>here</b> to see the answer.</summary>
  <p>The correct answer is B - Naive Bayes assumes that the probability of finding a certain word is independent from the probability of finding another word. So, for example, in the domain of movie reviews, it assumes that the probabilities of finding the words <code>Indiana</code> and <code>Jones</code> are not correlated, even if in practice we know that this is not the case.</p>

</details> 


# Python for Computational Linguists 2.1: Implementing Naive Bayes

## Introduction

Welcome to module 2.1! In this module we will review Naive Bayes and we will write our own implementation of the algorithm. We will use it to classify some toy examples, and then we will turn to Sentiment Analysis, by training our simple model on a research dataset. We will also introduce some new concepts: the `random` module and pseudo-random number generators, cross-validation, and some new functions from `numpy`.

### Naive Bayes refresher

> Note: this section borrows heavily from the Naive Bayes chapter of the lecture notes. Please refer to them or to the J&M for more details.

Naive Bayes is a simple classifier based on two assumptions:
- The **bag-of-words** assumption: word ordering doesn't matter. We represent each document in our dataset as a list of pairs $(word_i,frequency_i)$.
- The **conditional independence** assumption: the probability of one word appearing in a sentence is by no means correlated to the occurrence of another word.

These two assumptions heavily simplify our model, since they completely disregard grammar and and any domain knowledge (e.g. the `Indiana Jones` example in the pre-module quiz); but as we know now, they allow us to build a surprisingly efficient classifier.

Let's quickly go through the maths of Naive Bayes. Remember that given a document $d$ and a set of classes $C$, we need to assign the document to the class $\hat{c} \in C$ which has the maximum *a posteriori probability*, i.e. where $\hat{c}$ is defined as follows:

$$ \hat{c} = \text{argmax}_{c \in C} P(c \mid d)$$

How do we calculate $\hat{c}$ efficiently? Well, the Bayes Rule tells us that

$$ P(c \mid d) = \frac{P(c) \ P(d \mid c)}{P(d)} $$

Allowing us to rewrite 

$$\begin{align} 
\hat{c} &= \text{argmax}_{c \in C} P(c \mid d) \\
        &= \text{argmax}_{c \in C} \frac{P(c) \ P(d \mid c)}{P(d)}
\end{align}$$

However, the probability $d$ is constant for each class $c$, hence we can remove it, leaving only:

$$
\hat{c} = \text{argmax}_{c \in C} 
    \underbrace{P(c)}_\text{prior}
    \underbrace{P(d \mid c)}_\text{likelihood}
$$

Where the $prior$ is the **prior probability** of the class $c$ and the $likelihood$ is the probability of finding $d$ given the class $c$.

Using words as features, we can represent $d$ as a list of words $w_1, \dots , w_n$, hence 

$$
\hat{c} = \text{argmax}_{c \in C} 
    \underbrace{P(c)}_\text{prior}
    \underbrace{P(w_1, \dots , w_n \mid c)}_\text{likelihood} 
$$

However, $P(w_1, \dots , w_n \mid c)$ may be prohibitively hard to calculate, since we would need to estimate the probability of every possible combination of words. Here, the **conditional independence** assumption comes to the rescue, assuming that the probability of the words (i.e. features) are independent, allowing us to finally rewrite

$$
\begin{align}
\hat{c} &= \text{argmax}_{c \in C} 
       \underbrace{P(c)}_\text{prior}
       \underbrace{P(w_1 \mid c) \times \dots \times P(w_n \mid c)}_\text{likelihood} 
       \\
       &= \text{argmax}_{c \in C} 
       \underbrace{P(c)}_\text{prior}
       \underbrace{\prod_{w \in d}{P(w \mid c)}}_\text{likelihood} 
\end{align}
$$

What does all of this mean in practice? Well, that if we have a document $d$, all we need to know to classify it is:
- The *priors*, i.e. the probability of each class $c$
- The *likelihoods*, i.e. the probabilities for each word $w_i$ of the document to belong to each class $c$.

## Naive Bayes: a simple implementation

Let's begin with a simple example from the Post Lecture exercises (taken from J&M-3, exercise 4.2). Given the following short movie reviews, each labeled with a genre, either comedy or action:

| review                      | class  |
|-----------------------------|--------|
| fun, couple, love, love     | comedy |
| fast, furious, shoot        | action |
| couple, fly, fast, fun, fun | comedy |
| furious, shoot, shoot, fun  | action |
| fly, fast, shoot, love      | action |

And a new document D: 

| review                     | class  |
|----------------------------|--------|
| fast, couple, shoot, fly   | ?      |

We have to compute the most likely class for D.

Let's start by saving our documents in some vectors:

In [ ]:
train_docs = [
    ['fun', 'couple', 'love', 'love'],
    ['fast', 'furious', 'shoot'],
    ['couple', 'fly', 'fast', 'fun', 'fun'],
    ['furious', 'shoot', 'shoot', 'fun'],    
    ['fly', 'fast', 'shoot', 'love']]

train_labels = ['comedy', 'action', 'comedy', 'action', 'action']
test_doc = ['fast', 'couple', 'shoot', 'fly']

### Computing the priors

Remember what we needed to do? The first step is to compute the **priors**. Let's do that with a simple function:

In [ ]:
# What are our classes?
classes = set(train_labels)
print(classes)

# initialise the priors
priors = {}
for _class in classes:
    priors[_class] = 0

# count how many train example in each class
for _class in classes:
    for label in train_labels:
        if _class == label:
            priors[_class] += 1

print(priors)

> **<h3>💻 Try it yourself!</h3>**

Now the priors are not *normalised*, i.e. we have to bring each prior in the range $[0,1]$. Can you do that in the following cell?

In [ ]:
# write your code here

<hr>    <!-- please remember this! -->
<details>
  <summary>Click <b>here</b> to see the answer.</summary>
    <p><pre><code>
for _class in classes:
    priors[_class] = priors[_class] / len(train_labels)
    </code></pre></p>

</details> 

If you've got the priors correct, you should have $P(comedy) = 0.4$ and $P(action)=0.6$.

In [ ]:
print(priors)

> **<h3>💻 Try it yourself!</h3>**

Now let's wrap everything into a function. Can you complete the cell below?

In [ ]:
def compute_priors(labels):
    '''
    Computes the priors for a set of labels.
    '''

    # What are our classes?
    classes = set(labels)

    # -------------------------#
    #      E X E R C I S E     #
    # -------------------------#
    # initialise the priors
    priors = {}

    # ...?

    # count how many train example in each class
    
    # ...?
    
    # normalise the priors
    
    # ...?
    
    # ~      end exercise    ~ #
    
    return priors

<hr>    <!-- please remember this! -->
<details>
  <summary>Click <b>here</b> to see the answer.</summary>
  <p><pre><code>
def compute_priors(labels):
    '''
    Computes the priors for a set of labels.
    '''    
    # What are our classes?
    classes = set(labels)
    # initialise the priors
    priors = {}
    for _class in classes:
        priors[_class] = 0
    # count how many train example in each class
    for _class in classes:
        for label in labels:
            if _class == label:
                priors[_class] += 1 
    # normalise the priors
    for _class in classes:
        priors[_class] = priors[_class] / len(labels)        
    return priors
  </code></pre></p>
</details> 

In [ ]:
priors = compute_priors(train_labels)
print(priors)

### Find the vocabulary

Since we need to compute the likelihoods for all words in our vocabulary, we need to find all the words in our corpus first. Let's build our vocabulary with a simple function:

In [ ]:
def create_vocabulary(lines):
    '''
    Creates a vocabulary from test.
    
    Parameters
    ----------
    lines: a list of lists of words
    
    Returns
    -------
    a set with all the words in the lines.
    
    '''
    
    words = set()
    for line in lines:
        for word in line:
            words.add(word)
            
    return words

In [ ]:
create_vocabulary(train_docs)

### Computing the likelihoods

Now we need to compute the likelihoods of the words w.r.t. to each class. To do that, we can build a `dict` for each class, where `dict[word] = P(word|class)`. To do that we can slightly modify the function `create_vocab_dict` that we defined in [Module 1.4](../../module_1/module_1.4/module_1.4.ipynb). The function we defined was the following:

In [ ]:
def create_vocab_dict(lines):
    '''
    Collect vocabulary counts from text

    Parameters
    ----------
    f_processed_arg: a list of lists of words

    Returns
    -------
    a dictionary with words (str) as keys and counts(int) as values
    vocab={
    'SONNETS': 1
    }
    '''
    vocab={}# create an empty vocabulary dictionary to store words as keys and counts as values later. 
    for line in lines:
        for word in line:
            if word in vocab:
                vocab[word]+=1 # update the count for an existing word
            else:
                vocab[word]=1 # initilize the count for a new word
    return vocab

In [ ]:
print(create_vocab_dict(train_docs))

> **<h3>💻 Try it yourself!</h3>**

How can we modify this function to give us the likelihoods for each class? Modify the function below, directly derived from `create_vocab_dict`, to return the likelihoods instead of the raw counts.

In [ ]:
def compute_likelihoods(lines, vocabulary):
    '''
    Computes the likelihoods of words in a list of strings.

    Parameters
    ----------
    lines: a list of list of words
    vocabulary: the vocabulary of the full corpus

    Returns
    -------
    a dictionary with words (str) as keys and likelihoods(floats) as values
    vocab={
    'SONNETS': 0.01
    }
    '''
    
    
    # -------------------------#
    #      E X E R C I S E     #
    # -------------------------#
    # create an empty vocabulary dictionary to store words 
    # as keys and counts as values later. 
    
    likelihoods = {}
    
    # initialise the likelihoods
    # hint: iterate through the vocabulary and initialise
    # a new element of the likelihoods dict to 0

    
    
    # ~      end exercise    ~ #    
    
    # Now we iterate through the lines
    for line in lines:
        for word in line:
            likelihoods[word] += 1 

    # -------------------------#
    #      E X E R C I S E     #
    # -------------------------#
    # how long are our documents?
    total_tokens = 0
    
    # write your code here
    # (hint: sum the length of the lines in total_tokens!)
    
    for word in likelihoods:
        likelihoods[word] = # Write your code here
    
    # ~      end exercise    ~ #

    return likelihoods

<hr>    <!-- please remember this! -->
<details>
  <summary>Click <b>here</b> to see the answer.</summary>
    <p><pre><code>
def compute_likelihoods(lines, vocabulary):
    '''
    Computes the likelihoods of words in a list of strings.
    Parameters
    ----------
    lines: a list of list of words
    vocabulary: the vocabulary of the full corpus
    Returns
    -------
    a dictionary with words (str) as keys and likelihoods(floats) as values
    vocab={
    'SONNETS': 0.01
    }
    '''
    # -------------------------#
    #      E X E R C I S E     #
    # -------------------------#
    # create an empty vocabulary dictionary to store words 
    # as keys and counts as values later. 
    likelihoods = {}
    # initialise the likelihoods
    # hint: iterate through the vocabulary and initialise
    # a new element of the likelihoods dict to 0
    for word in vocabulary:
        likelihoods[word] = 0
    # ~      end exercise    ~ #    
    # Now we iterate through the lines
    for line in lines:
        for word in line:
            likelihoods[word] +=1 
    # -------------------------#
    #      E X E R C I S E     #
    # -------------------------#
    # how long are our documents?
    total_tokens = 0
    # write your code here
    # (hint: sum the length of the lines in total_tokens!)
    for line in lines:
        for word in line:
            total_tokens +=1    
    for word in likelihoods:
        likelihoods[word] = likelihoods[word]/total_tokens
    # ~      end exercise    ~ #
    return likelihoods
  </code></pre></p>
</details> 

In [ ]:
print(compute_likelihoods(train_docs, create_vocabulary(train_docs)))

Note that this method computes the likelihoods of the words in the whole vocabulary, irregardless of the classes. To get the likelihoods for each class we need to do as follows:

In [ ]:
target_class = 'action'
target_docs = []
vocabulary = create_vocabulary(train_docs)

# enumerate builds an (index, doc) list, hence allowing
# us to retrieve the label for each doc
for i, doc in enumerate(train_docs):
    if train_labels[i] == target_class:
        target_docs.append(doc)
        
print(target_docs)
print(compute_likelihoods(target_docs, vocabulary))

### The training function

> **<h3>💻 Try it yourself!</h3>**

Now we have all the instruments to build our training function. Can you complete it?

In [ ]:
def train_naive_bayes(documents, labels):
    
    classes = set(labels)
    
    # compute the priors
    priors = compute_priors(labels)
    vocabulary = create_vocabulary(documents)
    
    # this dict will contain the likelihoods, e.g.
    # likelihoods['action'] = {'fast': 0.2, 'furious': 0.1...
    likelihoods = {}
    
    # -------------------------#
    #      E X E R C I S E     #
    # -------------------------#
    for _class in classes:
        # get the documents belonging to each class:
        class_docs = []

        # put your code here
        for #...
        
        # compute the likelihood of this class
        likelihoods[_class] = # hint: use compute_likekihood defined above
        
    # ~      end exercise    ~ #
        
    return priors, likelihoods    

<hr>    <!-- please remember this! -->
<details>
  <summary>Click <b>here</b> to see the answer.</summary>
    <p><pre><code>
def train_naive_bayes(documents, labels):
    classes = set(labels)
    # compute the priors
    priors = compute_priors(labels)
    vocabulary = create_vocabulary(documents)
    # this dict will contain the likelihoods, e.g.
    # likelihoods['action'] = {'fast': 0.2, 'furious': 0.1...
    likelihoods = {}
    for _class in classes:
        # get the documents belonging to each class:
        class_docs = []
        for i, doc in enumerate(documents):
            if labels[i] == _class:
                class_docs.append(doc)
        # compute the likelihood of this class
        likelihoods[_class] = compute_likelihoods(class_docs, vocabulary)
    return priors, likelihoods 
    </code></pre></p>
</details> 

In [ ]:
priors, likelihoods = train_naive_bayes(train_docs, train_labels)

print('Priors:')
print(priors)

print('')
print('Likelihoods:')
print(likelihoods)

### Predict unknown classes

So now we have trained our model. How can we predict the likelihood of new sentences belonging to each class? Remember from above that

$$
\hat{c} = \text{argmax}_{c \in C} 
       \underbrace{P(d)}_\text{prior}
       \underbrace{\prod_{w \in d}{P(w \mid c)}}_\text{likelihood} 
$$

So, to predict the class of our new sentence `fast, couple, shoot, fly`, we need to:
- for each class `c`, we need to calculate `prob_c` by
    - multiplying the probability of each word `w` given that class `c`

Then, we'll just have to look at the maximum of our `prob_c`s. So, can we predict the class of `fast, fun, love, fly`? Let's write a function to do this.

In [ ]:
def bayes_predict(document, priors, likelihoods):
    '''
    Predicts the label for a document given the trained
    priors and likelihoods.
    
    Parameters
    ----------
    document: the document to analyse
    priors: the trained priors
    likelihoods: the trained likelihoods
    
    Return
    ------
    The probability for each class.
    '''
    
    classes_probabilities = {}

    # unpack the dictionary and iterate 
    # through the priors
    for label, prior in priors.items():
        
        # initialise the probability of a class to its prior
        prob_class = prior
        for word in document:
            if word in likelihoods[label]:
                # multiply the prior for the likelihood of each word
                prob_class = prob_class*likelihoods[label][word]
        classes_probabilities[label] = prob_class
        
    return classes_probabilities
    

Good! Let's try it with:

In [ ]:
document = ['fast', 'fun', 'love', 'fly']
bayes_predict(document, priors, likelihoods)

So for this document, `comedy` is the most likely class! 

Now let's try with the document for the assignment:

In [ ]:
print(test_doc)
bayes_predict(test_doc, priors, likelihoods)

Now the classes are both zero! How come? Well, we didn't apply any smoothing, so obviously at some point we are multiplying the likelihoods by zero, since $P(couple \ \mid action) = 0$ and $P(shoot \mid love) = 0$.

### Adding 1-smoothing

> **<h3>💻 Try it yourself!</h3>**

How can we modify `compute_likelihoods` to add 1-smoothing? Please update the function below.

Note that:
- We added a parameter (`smoothing`) to select the smoothing mode
- You will need to change how to normalise the likelihoods.

In [ ]:
def compute_likelihoods(lines, vocabulary, smoothing):
    '''
    Computes the likelihoods of words in a list of strings.

    Parameters
    ----------
    lines: a list of list of words
    vocabulary: the vocabulary of the training corpus
    smooething: the smoothing method to use

    Returns
    -------
    a dictionary with the probability for each class
    '''
    
    likelihoods = {}

    # populate the likelihoods
    for word in vocabulary:
        likelihoods[word] = 0
 

    # Now we iterate through the lines to count
    # the appearances of each word
    for line in lines:
        for word in line:
            likelihoods[word] +=1 

    # how long are our documents?
    total_tokens = 0
    for line in lines:
        for word in line:
            total_tokens +=1

    # Apply smoothing, if needed
    for word in likelihoods:
        if smoothing == 'none':
            
            for line in lines:
                for word in line:
                    total_tokens +=1

            for word in likelihoods:
                likelihoods[word] = likelihoods[word]/total_tokens
            
        elif smoothing == 'add1':
            # -------------------------#
            #      E X E R C I S E     #
            # -------------------------#
            # calculate the smoothing parameter for each word.
            smoothing_param = # ...?
            
            likelihoods[word] = # ...?
            # ~      end exercise    ~ #
        else:
            print('Unknown smoothing!')
            return

    return likelihoods

<hr>    <!-- please remember this! -->
<details>
  <summary>Click <b>here</b> to see the answer.</summary>
    <p><pre><code>
def compute_likelihoods(lines, vocabulary, smoothing):
    '''
    Computes the likelihoods of words in a list of strings.
    Parameters
    ----------
    lines: a list of list of words
    vocabulary: the vocabulary of the training corpus
    smooething: the smoothing method to use
    Returns
    -------
    a dictionary with the probability for each class
    '''
    likelihoods = {}
    # populate the likelihoods
    for word in vocabulary:
        likelihoods[word] = 0
    # Now we iterate through the lines to count
    # the appearances of each word
    for line in lines:
        for word in line:
            likelihoods[word] +=1 
    # how long are our documents?
    total_tokens = 0
    for line in lines:
        for word in line:
            total_tokens +=1
    # Apply smoothing, if needed
    for word in likelihoods:
        if smoothing == 'none':       
            for line in lines:
                for word in line:
                    total_tokens +=1
            for word in likelihoods:
                likelihoods[word] = likelihoods[word]/total_tokens
        elif smoothing == 'add1':
            # -------------------------#
            #      E X E R C I S E     #
            # -------------------------#
            # calculate the smoothing parameter for each word.
            smoothing_param = total_tokens + len(vocabulary)
            likelihoods[word] = (likelihoods[word] + 1)/smoothing_param
            # ~      end exercise    ~ #
        else:
            print('Unknown smoothing!')
            return
    return likelihoods
    </code></pre></p>

</details> 

Now let's update `train_naive_bayes` to instruct it to use smoothing, and let's see if the results we obtain are correct:

In [ ]:
def train_naive_bayes(documents, labels, smoothing):

    classes = set(labels)

    # compute the priors
    priors = compute_priors(labels)
    vocabulary = create_vocabulary(documents)

    # this dict will contain the likelihoods, e.g.
    # likelihoods['action'] = {'fast': 0.2, 'furious': 0.1...
    likelihoods = {}

    for _class in classes:
        # get the documents belonging to each class:
        class_docs = []

        for i, doc in enumerate(documents):
            if labels[i] == _class:
                class_docs.append(doc)

        # compute the likelihood of this class
        likelihoods[_class] = compute_likelihoods(class_docs, vocabulary, smoothing)

    return priors, likelihoods 

In [ ]:
priors, likelihoods = train_naive_bayes(train_docs, train_labels, 'add1')

print('Priors:')
print(priors)

print('')
print('Likelihoods:')
print(likelihoods)

In [ ]:
bayes_predict(test_doc, priors, likelihoods)

Great! We successfully implemented Naive Bayes in Python. 

### Improving Naive Bayes: `argmax` and log-likelihoods

What we've done until now is good - but there are still a couple of open issues. 
- First, we should note that the original function of Naive Bayes is based on $\text{argmax}$: how do we implement it in Python?
- Then, we may note that the probabilities returned by our model are very small (i.e. in the order of $10^{-5}$), which means that for big vocabularies we may reach probabilities close to zero.

While the latter may not sound like a big issue for you, unfortunately dealing with very small numbers may be problematic for a computer - as mentioned in Module 1.2, computers use [floating-point arithmetic](https://en.wikipedia.org/wiki/Floating-point_arithmetic), which in practice means that they aren't actually very good at dealing with very small or very big numbers.

Actually, machines are not very good at representing any number that can't be expressed as a sum of powers of $2$. Take for example $0.1$. If we ask Python to show it, it will print

In [ ]:
0.1

Which looks fine. But what if we ask Python to show the first 40 decimal digits of $0.1$? That shouldn't make sense, right? Let's see:

In [ ]:
print(f'{0.1:.40f}')

See? Under the hood Python represents 0.1 as the infinitely repeating binary fraction `0.0001100110011001100110011001100110011001100110011...`. Since we stop at 32 bits, we won't be able to exactly represent $0.1$, but we will get an approximation, i.e. the value printed in the cell above.

This does not mean that our code is buggy or that Python is getting it wrong. It's just how computers work! Unfortunately, this means that, since Naive Bayes will generate very small numbers, it will be more prone to error. For this reason, we use the **log-likelihoods** instead of the actual likelihoods, i.e. we will rewrite:
$$
\begin{align}
\hat{c} &= \text{argmax}_{c \in C} 
       \underbrace{P(c)}_\text{prior}
       \underbrace{\prod_{w \in d}{P(w \mid c)}}_\text{likelihood} \\
       &= \text{argmax}_{c \in C} \log(  
       \underbrace{P(c)}_\text{prior}
       \underbrace{\prod_{w \in d}{P(w \mid c)}}_\text{likelihood} ) \\
       &= \text{argmax}_{c \in C} 
       \underbrace{\log P(c)}_\text{prior} +
       \underbrace{\sum_{w \in d}{\log P(w \mid c)}}_\text{likelihood} 
\end{align}
$$

To do that, we will use `numpy`, arguably Python's most popular mathematical library, which offers the function `log`.

In the next cells we'll install numpy (if it's not already installed), and the we'll import it. You'll almost always find numpy imported as `np` - so if in any Python code you see something like `np.` you can assume that it's based on numpy.

In [ ]:
!pip install numpy

In [ ]:
# import and test with ln(e), i.e. the logarithm to the base e of e
import numpy as np
np.log(np.e)

Numpy also offers a convenient `argmax` function. Remember that `argmax` works by selecting the *index* of the biggest element of a list:

In [ ]:
np.argmax([0,-2,10,3])   # Remember that we use 0-based indexing!

Now can wrap everything in an updated `bayes_predict_log` function, which will be the log-based version of `bayes_predict`. Can you update it yourself?

> **<h3>💻 Try it yourself!</h3>**

In [ ]:
def bayes_predict_log(document, priors, likelihoods):
    '''
    Predicts the label for a document given the trained
    priors and likelihoods.
    
    Parameters
    ----------
    document: the document to analyse
    priors: the trained priors
    likelihoods: the trained likelihoods
    
    Return
    ------
    A tuple (best_class, probabilities), where the first element
    is the name of the best class, and the second element is the dictionary of
    the computed probabilities.
    '''
    
    classes_probabilities = {}

    # unpack the dictionary and iterate 
    # through the priors
    for label, prior in priors.items():
        
        # -------------------------#
        #      E X E R C I S E     #
        # -------------------------#
        # initialise the probability of a class to the log of its its prior
        prob_class = # ...?
        for word in document:
            if word in likelihoods[label]:
                # sum the prior with the log-likelihood of each word
                prob_class =  # ...?
        classes_probabilities[label] = prob_class
    
    # get the names of the classes
    class_names = list(priors.keys())

    # -------------------------#
    #      E X E R C I S E     #
    # -------------------------#
    # complete the next line by selecting the name of the best class.
    # hint: the probabilities are saved in the dictionaries, and
    # you can access them by using classes_probabilities.values().
    # note that you can use more than 1 line if you need!
    
    best_class = # ...?

    # ~      end exercise    ~ #
    
    return best_class, classes_probabilities    

<hr>    <!-- please remember this! -->
<details>
  <summary>Click <b>here</b> to see the answer.</summary>
  <p><pre><code>
def bayes_predict_log(document, priors, likelihoods):
    '''
    Predicts the label for a document given the trained
    priors and likelihoods.
    Parameters
    ----------
    document: the document to analyse
    priors: the trained priors
    likelihoods: the trained likelihoods
    Return
    ------
    A tuple (best_class, probabilities), where the first element
    is the name of the best class, and the second element is the dictionary of
    the computed probabilities.
    '''
    classes_probabilities = {}
    # unpack the dictionary and iterate 
    # through the priors
    for label, prior in priors.items():   
        # -------------------------#
        #      E X E R C I S E     #
        # -------------------------#
        # initialise the probability of a class to the log of its its prior
        prob_class = np.log(prior)
        for word in document:
            if word in likelihoods[label]:
                # sum the prior with the log-likelihood of each word
                prob_class = prob_class + np.log(likelihoods[label][word])
        classes_probabilities[label] = prob_class
    # get the names of the classes
    class_names = list(priors.keys())
    # -------------------------#
    #      E X E R C I S E     #
    # -------------------------#
    # complete the next line by selecting the name of the best class.
    # hint: the probabilities are saved in the dictionaries, and
    # you can access them by using classes_probabilities.values(), which
    # can be converted to a list using list(x)
    best_class = class_names[np.argmax(list(classes_probabilities.values()))]
    # ~      end exercise    ~ #
    return best_class, classes_probabilities          
  </code></pre></p>

</details> 

In [ ]:
document = ['fast', 'couple', 'shoot', 'fly']
bayes_predict_log(document, priors, likelihoods)

Good! Now the values are more reasonable and we can expect our computer to make less errors. Also, the code will run quicker, as summing typically requires less computational time than multiplying.

### Other improvements

Naturally, we could improve our Naive Bayes implementation even further. We could try different smoothing techniques, we could cache the log-likelihoods in the `likelihoods` dictionary instead of applying `np.log` every time, and so on. For now, let's just be happy with what we've done, and let's apply our code to a real world dataset.

## Classify the *Thumbs up?* dataset

We will use the data from the paper [*Thumbs up? Sentiment Classification using Machine Learning Techniques*](https://www.aclweb.org/anthology/W02-1011.pdf). The dataset consists in 1301 positive reviews and 752 negative reviews from [IMDb](https://www.imdb.com/), a large online database of facts and reviews of movies.

For this part, you are **not** required to understand how the data loading and preparation works, but we encourage you to study the code anyway. After we have loaded the dataset, you will use the model you just wrote to classify it. 
The paper reports 78.7 accuracy for their Naive Bayes unigrams model; let's see if your code is able to reach this performance.

### Data preprocessing

In this section, we will download the data from GitHub and we'll preprocess them so that they are in the same format as above. As a reminder, training data is formatted this way:

```python
train_docs = [['fun', 'couple', 'love', 'love'],
 ['fast', 'furious', 'shoot'],
 ['couple', 'fly', 'fast', 'fun', 'fun'],
 ['furious', 'shoot', 'shoot', 'fun'],
 ['fly', 'fast', 'shoot', 'love']]

 train_labels = ['comedy', 'action', 'comedy', 'action', 'action']
```

Our goal is to transform the Thumb Up! dataset in this format in order to use
the functions we defined above without any modification.

Let's start by downloading the data. We use the UNIX utility `wget` for this; please be aware that if you're running this code on a Windows machine it most likely won't work, and you'll have to download the dataset by yourself and unzip it in the same folder of this notebook. If you're running this notebook locally or on Binder, you may skip this cell.

In [ ]:
!wget https://github.com/cambridgeltl/python4cl/raw/master/module_2/module_2.2/data.zip
!unzip -n -q data.zip -d ../module_2.2/data

Let's now load the data. We will define a function that:

1. Finds all the files in a directory;
2. For each file:
  - Loads the content of the file in a string, and
  - Extracts all the words from the document by using `re.findall`;
3. Finally, it returns the data as a list of lists of words, e.g.:

```python
[['hello', 'world', '...' ],   # document 1
 ['lorem', 'ipsum', '...' ],   # document 2
 ...
]
```

You should rememeber `re.findall` from Module 1, but we encourage you to have a look at the [documentation](https://docs.python.org/3/library/re.html#re.findall)
and play with this function.

In [ ]:
# modules needed to locate and split the data
import os
import random
import re


def process_docs(directory):
    """
    Parameters
    ----------
    directory: a directory containing positive/negative samples from the Thumbs
    Up! dataset.

    Return
    ------
    A list of of documents, where each document is a list of words.
    """
    
    
    docs=[] # this will contain the documents we'll find

    # walk through all files in the folder
    for filename in os.listdir(directory):

        # skip files that do not have the right extension
        if not filename.endswith(".txt"):
            continue
        
        # Load the document:
        # open the file as read only
        with open(os.path.join(directory,filename), 'r') as f:
          # read all text and put it in a single long string
          doc = ''.join(f.readlines())
          # extract all the words; 
          # re.findall finds all the substrings matching a regular expression.
          # try the line below on some toy examples to understand how it works.
          doc = re.findall(r'\w+', doc)

        # add the new document to the list of documents
        docs.append(doc)

    # return everything we've found
    return docs
 
# load positive examples
positive_docs = process_docs(os.path.join('..', 'module_2.2', 'data', 'pos'))
# load negative examples
negative_docs = process_docs(os.path.join('..', 'module_2.2', 'data', 'neg'))

Now we have our lists of documents:

In [ ]:
print('First 10 words of the first 2 positive documents:')
print(positive_docs[0][:10])
print(positive_docs[1][:10])
print('First 10 words of the first 2 negative documents:')
print(negative_docs[0][:10])
print(negative_docs[1][:10])

Now we need to shuffle this data to obtain a training and testing set. We will 
use [`zip()`](https://docs.python.org/3/library/functions.html#zip), which 
allows us to create two list of tuples `[(positive_doc_1, 'pos'), ... (positive_doc_n, 'pos')]`
and `[(negative_doc_1, 'neg'), ... (negative_doc_n, 'neg')]` for positive and 
negative documents respectively.

In [ ]:
all_docs = list(zip(positive_docs, ['pos'] * len(positive_docs))) +\
  list(zip(negative_docs, ['neg'] * len(negative_docs)))

> **<h3>💻 Try it yourself!</h3>**

Have a look at the list `all_docs` in the cell below. Reassure yourself that the data is in the 
format defined above; check that the first element has a `pos` label and that the last element has a `neg` label.

### Shuffle the data

Now we need to shuffle the data. We will use the function [`random.shuffle()`](https://docs.python.org/3/library/random.html#random.shuffle), which shuffles an array using a [pseudorandom number generator](https://en.wikipedia.org/wiki/Pseudorandom_number_generator). You don't really need to know the nuts and bolts
of random number generation, but it's very important to stress that in computer
science, random number are not actually that random, but they are generated by
a function that emulates a random distribution.

This function starts from a value, called seed. If we manually set the seed, we 
can ensure that the data generated by our random function is consistent 
throughout different runs, hence guaranteeing the repetibility of our experiments.

> 💻 Try it yourself!

Look at the cell below and run it. Then, change the parameter of `random.seed()` with a value of your choice and then run it again. Now change it again to the original value (`1203`). What happens? Why? Have a very quick look at the 
Wikipedia page of  pseudorandom number generators using the link above. 

In [ ]:
random.seed(1203)
print(random.randrange(100))  # generates a random number between 0 and 100
print(random.randrange(100))  
print(random.randrange(100))  

Now that you understand how pseudo random number generation works, let's shuffle
the array. Note that here we instantiate a new `Random` with seed $0.42$.

In [ ]:
random.Random(.42).shuffle(all_docs)

Retain the first 70% of the documents for training, and the remaining 30% for testing:

In [ ]:
train_docs_thumbsup = all_docs[:int(len(all_docs)*(0.7))]
test_docs_thumbsup =all_docs[int(len(all_docs)*(0.7)):]

Now, extract the documents and the labels:

In [ ]:
train_txts_thumbsup = [txt for txt, _ in train_docs_thumbsup]
train_labels_thumbsup = [label for _, label in train_docs_thumbsup]

Is the array really random? Let's have a look at the first 10 labels. Now they shouldn't be all positives:

In [ ]:
train_labels_thumbsup[:10]

### Run the model

Now we have our documents and labels arrays in the same format as before. Let's feed them to our `train_naive_bayes()` function and train the model.

In [ ]:
tu_priors, tu_likelihoods = train_naive_bayes(train_txts_thumbsup, train_labels_thumbsup, 'add1')

> **<h3>💻 Try it yourself!</h3>**

Very well! Now let's see how well our classifier works. In the cell below, let's
calculate the accuracy of our algorithm.

Accuracy is a metric that calculates how many times the algorithm returns the correct result. Update the cell below to obtain the accuracy of our Naive Bayes classifier on the test set.

In [ ]:
correct_answers = 0
# iterate over the documents in the test set;
# unpack each (document, label) tuple
for doc, true_label in test_docs_thumbsup:
  # predict the label for the document
  predicted_label, _ = bayes_predict_log(doc, tu_priors, tu_likelihoods)
  # if the predicted label is the same as the true label, 
  # update the counter
  # write your code here!

# calculate the accuracy score
accuracy = # write your code here!

# print the accuracy
print(accuracy)

<hr>    <!-- please remember this! -->
<details>
  <summary>Click <b>here</b> to see the answer.</summary>
    <p><pre><code>correct_answers = 0
# iterate over the documents in the training set;
# unpack each (document, label) tuple
for doc, true_label in test_docs_thumbsup:
  # predict the label for the document
  predicted_label, _ = bayes_predict_log(doc, tu_priors, tu_likelihoods)
  # if the predicted label is the same as the true label, 
  # update the counter
  if true_label == predicted_label:
    correct_answers += 1
# calculate the accuracy score
accuracy = correct_answers/len(test_docs_thumbsup)
# print the accuracy
print(accuracy)
    </code></pre></p>
</details> 

Great! $78.\overline{6}\%$ accuracy with such a simple model. The original paper obtained 
$78.7\%$ with a similar configuration; however, we must note that their score is obtained by averaging the score of the models with three different training/test split, and they used further preprocessing steps, so the scores are not directly comparable. 

However, your score is very similar to the one obtained in the paper, which is 
very good! In the next module we will use another popular Python library, `scikit-learn`, to try and improve this result.

# ✍️ Final Assessment

The final test is aimed at obtaining the best performance out of our Naive Bayes model with little effort. You will see some advanced techniques in the next module, such as using bi-grams instead of unigrams; for now, let's focus on simpler tactics to improve our results that every machine learning practitioner should know.

1. In the first part of the final test you will learn what cross-validation is, and why is it useful to build more robust models. 
2. In the second part of your final test you will experiment with *hyperparameter tuning*: you will try different values of the smoothing parameter $\alpha$ to see how it affects the results of our algoritm.

## ✍️ Part 1 : Cross-Validation

When developing machine learning models, a common problem is to tune your algorithm too heavily to perform well on a specific set of data; this problem is called **overfitting**. This happens because the model tries too hard to obtain good performance on a specific subset of your data, so it is not able to **generalise** well anymore on unseen data.

In our case, we are using a training set and a test set. Even if we tune our Naive Bayes implementation very well on our test set, we have no clue on how it would perform on new, unseen data. 

There are several solutions to this problem; with neural networks, a typical solution is to have three sets: a training, development (or validation), and training set; we train on the training set, trying to obtain the best performance on the development set. When we are satisfied with the performance of our algorithm on the dev set, we run our model on the yet-unseen test set; since we never ran our model on this set, we will have a good idea on its ability to generalise on unseen data.

However, this is not always feasible. If you don't have enough data, for example, an alternative is to use **cross-validation**. In cross-validation, you divide your corpus in a $k$ subsets, called *folds*; then, for each of the $k$ folds, we:

- train the model on the other $k-1$ folds
- evaluate the model on the current fold.

We then report the average accuracy obtained on the different folds. This technique is called $k$-fold cross-validation, and it's helpful to verify that our model is able to perform well on different training/test splits, without relying to much on an arbitrary train/test split which may result in an abnormally good (or bad) model just because of chance.

To learn more about cross-validation, have a look at the [documentation of scikit-learn](https://scikit-learn.org/stable/modules/cross_validation.html).

In this exercise, you will have to implement 5-fold cross-validation. Let's start by defining the number of folds $k$ and the seed we'll use for our experiment ([42](https://en.wikipedia.org/wiki/Phrases_from_The_Hitchhiker%27s_Guide_to_the_Galaxy#Answer_to_the_Ultimate_Question_of_Life,_the_Universe,_and_Everything_(42))).

In [ ]:
num_folds = 5
cross_val_seed = 42

Remember that we have our documents in `all_docs`. We need to split the array in `num_folds` different subsets, i.e., our folds.

We will use scikit-learn's `KFold` to do that. This helper class splits an array in $k$ folds for us, returning the indexes of the elements of each fold.
Let'see how it works in an example:

In [ ]:
# import KFold
from sklearn.model_selection import KFold

# test array
# KFold requires that our data is in a numpy array
# to work properly
X = np.array(['cat','dog','bird','cow','fish'])

# create the KFold object and create the splits
kf = KFold(n_splits=num_folds)
kf.get_n_splits(X)

# iterate over the splits and show them
i = 0
for train_index, test_index in kf.split(X):
  i += 1
  print(f'Fold {i}')
  print(X[train_index])
  print(X[test_index])

Good! Now, let's repeat the same process on our data. In the cell below, please:
- Create a `KFold` object. Looking at the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html), enable shuffling and use the `cross_val_seed` we defined above as seed.
- Ask `KFold` to generate the splits, using `KFold.get_n_splits()`.

> Please note that if you use different seed values here or in the previous sections of the notebooks your results may differ.

In [ ]:
# -------------------------#
#      E X E R C I S E     #
# -------------------------#
# create the KFold object
kf = KFold( # ...
# create the splits
kf.get_n_splits(all_docs)

Great! Now let's see if we obtained the same result. Run the following cell:

In [ ]:
for train_index, test_index in kf.split(all_docs):
    print(train_index)
    break

The cell above **must** print ```[   0    1    2 ... 1997 1998 1999]```.

Now let's run the model five times and compute the average accuracy over the runs. Complete the following cell by saving the accuracy of each fold in the list `accuracy_list`.

In [ ]:
accuracy_list = []


for fold_idx, (train_index, test_index) in enumerate(kf.split(all_docs)):

    print(f'Running fold {fold_idx+1}...')
    
    # Get the current folds
    fold_train = np.array(all_docs)[train_index]
    fold_test = np.array(all_docs)[test_index]

    # unpack sentences and labels
    fold_txts = [txt for txt, _ in fold_train]
    fold_labels = [label for _, label in fold_train]

    # -------------------------#
    #      E X E R C I S E     #
    # -------------------------#
    # run the model and save it

    fold_priors, fold_likelihoods = # ... complete this line

    # ~      end exercise    ~ #
    # -------------------------#
    #      E X E R C I S E     #
    # -------------------------#
    # compute the fold's accuracy and append it in the
    # list 'accuracy_list'
    # hint: you can mostly reuse the code for the Thumbs Up! dataset
    
    correct_answers = 0

    # iterate over the documents in the test set (fold_test)
    # write your code here!

      
    # calculate the accuracy score
    accuracy = # ...

    # ~      end exercise    ~ #
    # print the accuracy
    accuracy_list.append(accuracy)

print('Done!')

Let'a check our results:

In [ ]:
print(f'Mean accuracy      : { np.mean(accuracy_list):.5f}')
print(f'Standard deviation : { np.std(accuracy_list):.5f}')

If you've done everything right, the cell above should print:
```
Mean accuracy      : 0.81450
Standard deviation : 0.00992
```

Very well! Now you've learned what cross-validation is and why is it important. Let's go on with hyperparameter tuning.

## ✍️ Part 2: Tuning $\alpha$

Let's continue playing with the model. Now we know that the code we wrote is quite stable, since $k$-fold cross validation shows that we obtain good results even using different training/testing splits.

To improve the model a little more, we could test if changing some of its internal parameters results in an improvement. For Naive Bayes, the main parameter that we can change is the smoothing parameter $\alpha$; for example, we may want to test what happens if, instead of 1-smoothing (i.e. $\alpha = 1$) we use $\alpha = 0.5$, or $\alpha = 0.1$, or $\alpha = 0.01$. This process is called **hyperparameter tuning**, and it's very important in modern machine learning; for example, with neural networks, once you have defined your basic architecture you will want to try what happens if you add more neurons to your network, if you change *activation function*, and so on.

For now, let's concentrate on Naive Bayes. Please update `compute_likelihoods` to use a custom parameter $\alpha$ in the cell below.

In [ ]:
def compute_likelihoods(lines, vocabulary, alpha):
    '''
    Computes the likelihoods of words in a list of strings.
    Parameters
    ----------
    lines: a list of list of words
    vocabulary: the vocabulary of the training corpus
    alpha: the alpha value to use for smoothing.
    Returns
    -------
    a dictionary with the probability for each class
    '''
    likelihoods = {}
    # populate the likelihoods
    for word in vocabulary:
        likelihoods[word] = 0
    # Now we iterate through the lines to count
    # the appearances of each word
    for line in lines:
        for word in line:
            likelihoods[word] +=1 
    # how long are our documents?
    total_tokens = 0
    for line in lines:
        for word in line:
            total_tokens +=1
    # Apply smoothing, if needed
    for word in likelihoods:
          # -------------------------#
          #      E X E R C I S E     #
          # -------------------------#
          # calculate the smoothing parameter for each word.
          smoothing_param = total_tokens + len(vocabulary)
          likelihoods[word] = # write your code here
          # ~      end exercise    ~ #
    return likelihoods

Very well. Now let's see what happens if we call `train_naive_bayes` with different smoothing values. Run the cell below and see what happens; then, update the smoothing parameter `.1` to other values and see how `priors` and `likelihoods` change.

In [ ]:
priors, likelihoods = train_naive_bayes(train_docs, train_labels, .1)

print('Priors:')
print(priors)

print('')
print('Likelihoods:')
print(likelihoods)

Now it's time to try different smoothing parameters on the Thumbs Up! dataset. In the cell below, you have to:

- Instantiate a list with the smoothing parameters we want to try; we will use $1, 0.5, 0.1, 0.01$ and an empty dictionary that will contain the accuracies for each alpha.
- Write a `for` loop that for each $\alpha$, 
  - runs $k$-fold cross validation on the splits defined above, and
  - saves the average accuracy for every $\alpha$ in a dictionary.

In [ ]:
# -------------------------#
#      E X E R C I S E     #
# -------------------------#
# define the alphas we will use for the experiment
# and initialise the dictionary that will hold 
# the results

alphas = # ...
alphas_accuracy = {}

# ~      end exercise    ~ #


for alpha in alphas:

  accuracy_list = []


  for fold_idx, (train_index, test_index) in enumerate(kf.split(all_docs)):

      print(f'Running alpha = {alpha}, fold {fold_idx+1}...')
      
      # Get the current folds
      fold_train = np.array(all_docs)[train_index]
      fold_test = np.array(all_docs)[test_index]

      # unpack sentences and labels
      fold_txts = [txt for txt, _ in fold_train]
      fold_labels = [label for _, label in fold_train]

      # -------------------------#
      #      E X E R C I S E     #
      # -------------------------#
      # run the model and save it

      fold_priors, fold_likelihoods = # ...

      # ~      end exercise    ~ #
      # -------------------------#
      #      E X E R C I S E     #
      # -------------------------#
      # compute the fold's accuracy 
      
      correct_answers = 0

      # evaluate the model
      
      # calculate the fold's accuracy score
      accuracy = correct_answers/len(fold_test)

      # ~      end exercise    ~ #
      # save the accuracy
      accuracy_list.append(accuracy)

  # closing for fold_idx, (train_index, test_index) in enumerate(kf.split(all_docs)):
  
  # -------------------------#
  #      E X E R C I S E     #
  # -------------------------#
  # save the average accuracy of the run in alphas_accuracy
  
  alphas_accuracy[alpha] = # ...

  # ~      end exercise    ~ #

print('Done!')
print('Results:')
print(alphas_accuracy)

> Hint: to double check that what you wrote is correct, for $\alpha = 1$ you should obtain the same results you obtained in the cross-validation exercise.

Very well! If you've done everything correct, you should have noticed that decreasing values of alpha correspond to decreasing performance, so $\alpha = 1$ is indeed the best smoothing parameter. 

# Wrapping up

In this notebook you should have learned how to write a simple model, how to evaluate it, and how to try to tune it to obtain the best possible performance. In the next model we will try to improve the model further by using more complex features, such as bigrams.

### Additional resources

- Naive Bayes chapter from [Manning, Raghavan and Schütze's Introduction to Information Retrieval](https://nlp.stanford.edu/IR-book/html/htmledition/naive-bayes-text-classification-1.html)
- Python's official documentation on [floating-point arithmetics and its issues](https://docs.python.org/3/tutorial/floatingpoint.html)
-  `scikit-learn`'s [implementation of Naive Bayes](https://github.com/scikit-learn/scikit-learn/blob/0fb307bf3/sklearn/naive_bayes.py#L669)
- Zhai and Lafferty, *A study of smoothing methods for language models applied to information retrieval*: a paper comparing different smoothing techniques for language models. [[pdf](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.58.8978&rep=rep1&type=pdf)]